In [14]:
install.packages("kknn")
install.packages("GGally")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

also installing the dependencies ‘ggplot2’, ‘ggstats’


Warning message in install.packages("GGally"):
“installation of package ‘ggplot2’ had non-zero exit status”
Warning message in install.packages("GGally"):
“installation of package ‘ggstats’ had non-zero exit status”
Warning message in install.packages("GGally"):
“installation of package ‘GGally’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [13]:
### Run this cell before continuing.
library(tidyverse)
library(repr)
library(tidymodels)
library(GGally)
options(repr.matrix.max.rows = 6)

ERROR: Error: package or namespace load failed for ‘tidyverse’ in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 there is no package called ‘ggplot2’


# Predicting The Occurrence of Exercised Induced-Angina by Analysis of Age and Maximum Heart Rate
Angelyca  P, Jay P, Elaine S 

# Introduction: 

### Background Information: 

Angina is categorized as a type of chest pain due to a reduced blood flow to the heart. Specific to this project, exercise induced angina is being analyzed and predicted. Exercise induced angina is a distinct class of angina, as the symptoms only occur when triggered by exercise. When the body is at rest, there is a lower demand of oxygen, allowing the heart to properly function despite the limiting amount of blood flow, with the absence of angina symptoms. Whereas, during exercise, the body will have an increased demand of oxygen, and thus blood flow. However, in this case, the heart is unable to function with the lack of oxygen, without experiencing chest pain sensations, or angina symptoms. Studies have shown that increasing age is often accompanied with a greater risk of experiencing angina (chest pain). 

In this project, maximum heart rate and age of an individual will be the two predictors used in classifying an unknown observation into the category of whether an individual will experience exercise induced angina. 


### Predictive Question: Will the individual experience exercise induced angina? 


To conduct our data analysis, a number of different combinations of two predictors will be tested, where the best combination will be chosen and used to be the 
the variables/columns of age and the max rate will be used to make predictions about the occurrence of exercise induced angina. Classification of the occurrence of exercise induced angina will be done through the use of a knn classification model. 

In [ ]:
### Downloading raw dataset from the web and renaming columns. 

url <- "https://raw.githubusercontent.com/elaines6/dsci-100-group-project/main/processed.cleveland.data"

heart_raw_data <- read_csv(url, col_names = c("age", "sex", "chest_pain_type", "resting_bp", "chol", "fasting_bs", "rest_ecg", "max_rate",
                                             "exercise_induced_angina", "ST_depression", "slope_peak_exercise", "major_vessels", "thal", "diagnosis_heart_disease"))
heart_data_tidy <- heart_raw_data |>
    mutate(sex = as_factor(sex), chest_pain_type = as_factor(chest_pain_type), exercise_induced_angina = as_factor(exercise_induced_angina)) |>
    mutate(sex = fct_recode(sex, "Female" = "0", "Male" = "1")) |>
    mutate(exercise_induced_angina = fct_recode(exercise_induced_angina, "Yes" = "1", "No" = "0")) |>
    mutate(chest_pain_type = fct_recode(chest_pain_type, "Typical angina" = "1", "Atypical angina" = "2", "Non-anginal pain" = "3", "Asymptomatic" = "4")) |>
    filter(age != "?" & sex != "?"& chest_pain_type != "?"& resting_bp != "?"& max_rate!= "?"& exercise_induced_angina != "?")


#what is this part for?!! 
unscaled_heart_data <- heart_data_tidy |>
    select(resting_bp, max_rate, exercise_induced_angina) |>
    mutate(resting_bp = as.integer(resting_bp), max_rate = as.integer(max_rate))


#Checking if standardizing the data matters for plotting 
unscaled_heart_data_to_check <- heart_data_tidy |>
    select(resting_bp, age, exercise_induced_angina) |>
    mutate(resting_bp = as.integer(resting_bp), max_rate = as.integer(age))

# After deciding the which plot to use, we decided to use heart_plot_four, which is age and max_rate 
# heart_plot_four presents the best data to use for a classification question/model 
# used ggcorrplot to summarize our process of testing variation of combinations and analyzing which duo is the best on 

final_unscaled_heart_data <- heart_data_tidy |>
    select(age, max_rate, exercise_induced_angina) |>
    mutate(age = as.integer(age), max_rate = as.integer(max_rate)) 

final_unscaled_heart_data 

In [ ]:
### Testing to see if there are any NA/missing values from the dataset.

heart_raw_data[heart_raw_data == "?"] <- NA

heart_raw_data %>%
    map_df(~sum(is.na(.x)))

In [12]:
### The clean and tidy data set. 
heart_data_tidy

ERROR: Error in eval(expr, envir, enclos): object 'heart_data_tidy' not found


From this cell to plot below is for checking whether the standardized data matters for deciding the best predictors

In [ ]:
#
heart_recipe <- recipe(exercise_induced_angina ~ ., data = unscaled_heart_data_to_check) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors()) 
heart_recipe

In [ ]:
scaled_heart_rate <- bake(heart_recipe, unscaled_heart_data_to_check)
scaled_heart_rate

In [ ]:
heart_plot_testing <- scaled_heart_rate |>
    ggplot(aes(x = age, y = resting_bp, color = exercise_induced_angina))+ 
    geom_point(alpha = 0.5) +
    labs(x = "Age (standardized)", 
         y = "Resting Blood Pressure (standardized)",
         color = "Diagnosis of exercise induced angina") +
    scale_color_manual(values = c("orange2", "steelblue2")) + 
    theme(text = element_text(size = 15))

heart_plot_testing 

# can we get rid of this? what is the purpose of this graph ? is it just test if theres a difference between standardized and unstandardized 

In [ ]:
# Getting the number of samples in the plot 4, but do I need to upsample this??
final_unscaled_heart_data|>
    group_by(exercise_induced_angina)|>
    summarize(count= n())

In [ ]:
heart_plot_four <- heart_data_tidy |>
    ggplot(aes(x = age, y = max_rate, color = exercise_induced_angina)) + 
    geom_point(alpha = 0.5) +
    labs(x = "Age", 
         y = "Max Rate",
         color = "Diagnosis of exercise induced angina") +
    scale_color_manual(values = c("orange2", "steelblue2")) + 
    theme(text = element_text(size = 15))

heart_plot_four

In [ ]:
#Getting the average values of the each columns. This means that I cannot get the average of everything.  

predictors_of_heart_data <- heart_data_tidy
mean_of_columns <- colMeans(predictors_of_heart_data) 
mean_of_columns

frame_of_mean <- tibble( row.names=c("age", "sex", "chest_pain_type", "resting_bp", "chol", "fasting_bs", "rest_ecg", "max_rate",
                                             "exercise_induced_angina", "ST_depression", "slope_peak_exercise", "major_vessels", "thal", "diagnosis_heart_disease"), mean_of_columns)
frame_of_mean

In [ ]:
#Getting K value which has the highgest accuracy
heart_split<- initial_split(heart_data_tidy, prop= 0.75, strata= exercise_induced_angina)
heart_train<- training(heart_split)
heart_test<- testing(heart_split) 

In [ ]:
# Getting the proportion of the induced_angina in the training set 
heart_proportions <- heart_train |>
                      group_by(exercise_induced_angina) |>
                      summarize(n = n()) |>
                      mutate(percent = 100*n/nrow(heart_train))

heart_proportions

In [ ]:
#creating a recipe for predicting the class of exericse induced angina based on the two predictors of age and max rate 
heart_recipe <- recipe(exercise_induced_angina ~ age + max_rate, data = heart_train) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())

In [ ]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", 
                             neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")


In [ ]:
heart_vfold <- vfold_cv(heart_train, v = 10, strata = exercise_induced_angina)


k_vals <- tibble(neighbors = seq(from = 1, to = 50, by = 2))

knn_results <- workflow() |>
  add_recipe(heart_recipe) |>
  add_model(knn_spec) |>
  tune_grid(resamples = heart_vfold, grid = k_vals) |>
  collect_metrics() 

accuracies <- knn_results |>
  filter(.metric == "accuracy")

accuracies

accuracy_versus_k <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
                     geom_point() +
                     geom_line() +
                     labs(x = "Neighbors", y = "Accuracy Estimate") +
                     theme(text = element_text(size = 20)) +
                     scale_x_continuous(breaks = seq(0, 50, by = 2)) +
                     scale_y_continuous(limits = c(0.4, 1.0))
accuracy_versus_k

In [ ]:
final_heart_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 42) |>
       set_engine("kknn") |>
       set_mode("classification")

final_heart_fit <- workflow() |>
             add_recipe(heart_recipe) |>
             add_model(final_heart_spec) |>
            fit(data = heart_train)
final_heart_fit

In [ ]:
heart_predictions <- predict(final_heart_fit, heart_test) |>
                        bind_cols(heart_test)

heart_predictions

heart_metrics <- heart_predictions |> metrics(truth = exercise_induced_angina, estimate = .pred_class)
heart_conf_mat <- heart_predictions |>
                        conf_mat(truth = exercise_induced_angina, estimate = .pred_class) 
heart_metrics
heart_conf_mat

In [ ]:
False negative top right and false positive left bottom and true negative left top  true negative false negative is more serious. 

# Discussion 

#### Summarize your findings here!! 

#### What impact could such findings have? 
Such findings of this data analysis would be able to predict whether or not individuals will experience exercise induced angina, based on their maximum heart rate and age. 


#### What future questions could this lead to?
What other predictors could be used to accurately predict whether an individual will exercise induced angina ? Can exercise induced angina be a predictor for further underlying cardiac health issues (if an individual experiences exercise induced angina, are they at a higher risk for cardiac arrest, stroke, and other complications?)





### References: (cite these at the end) 

https://archive.ics.uci.edu/dataset/45/heart+disease


https://www.ahajournals.org/doi/10.1161/01.CIR.0000086898.96021.B9#:~:text=Pain%20in%20the%20left%20chest,more%20frequent%20with%20advancing%20age.
